<h1 style="background-color: green;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐תהליך הסבת חלקות (דו-ממדיות) בשטח לא-מוסדר לבנק"ל מודרני🌐

In [ ]:
print('14. Unregistered Parcels Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
RawData_Folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today = dt.date.today()
product_name = str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + product_name
del [today, product_name]

<h1 style="text-align:right;font-size:100%">
שכבת חלקות קיימת

In [ ]:
parcel_all_cols = ['Identifier', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'STATUS', 'LEGAL_AREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

parcel_all = gpd.read_file(RawData_Folder + 'ArcStorm\PARCEL_ALL.gdb',
                           layer          = 'PARCEL_ALL',
                           include_fields = parcel_all_cols)

parcel_all = parcel_all.to_crs(2039)

del [parcel_all_cols]

print('         input 1/3 read')

<h1 style="text-align:right;font-size:100%">
רשומות של תצ"ר בשטח לא מוסדר משכבת גבולות תהליכי קדסטר

In [ ]:
cpb_cols = ['CPBUniqueID', 'ProcessName', 'ProcessType']

cpb_layer = gpd.read_file(product_path,
                          layer = 'CadasterProcessBorders',
                          include_fields = cpb_cols,
                          ignore_geometry = True)

unreg_tazars = cpb_layer[cpb_layer['ProcessType'] == 11]


del [cpb_cols, cpb_layer]

print('         input 2/3 read')

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר מודרנית

In [ ]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path,
                       layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)

Blocks = Blocks.astype(int)


del[Blocks_cols, RawData_Folder, product_path]

print('         input 3/3 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  14.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
הסרת חלקות שמספרן הוא 0 או ריק וחלקות שמספר הגוש שלהן הוא 0 או ריק

In [ ]:
parcel_all = parcel_all[(parcel_all['PARCEL'] != 0)
                                    |
                        (~parcel_all['PARCEL'].isna())
                                    |
                        (parcel_all['GUSH_NUM'] != 0)
                                    |
                       (~parcel_all['GUSH_NUM'].isna())]

<h1 style="text-align:right;font-size:100%">
סינון שכבת חלקות לחלקות לא מוסדרות לפי שדה של סטאטוס

In [ ]:
registered_status = [1, 3, 6, 16]

unreg_parcels = parcel_all[~parcel_all['STATUS'].isin(registered_status)]
unreg_parcels.drop(columns = 'STATUS', inplace = True)

del [registered_status, parcel_all]

In [ ]:
print('  14.c Data conversion')

<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם תהליך ואיחוד מידע עם שכבת תצ"ר בשטח לא מוסדר

In [ ]:
unreg_parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = unreg_parcels[['TALAR_NUMBER', 'TALAR_YEAR']].fillna(0)
unreg_parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = unreg_parcels[['TALAR_NUMBER', 'TALAR_YEAR']].astype(int)

unreg_parcels['ProcessName'] = unreg_parcels['TALAR_NUMBER'].astype(str) + '/' + unreg_parcels['TALAR_YEAR'].astype(str)
unreg_parcels['ProcessName'] = unreg_parcels['ProcessName'].replace('0/0', np.nan)

unreg_parcels.drop(columns=['TALAR_NUMBER', 'TALAR_YEAR'], inplace = True)

unreg_parcels = unreg_parcels.merge(unreg_tazars, on = 'ProcessName', how = 'left')

<h1 style="text-align:right;font-size:100%">
יצירת שדה של מספר גוש מלא ואיחוד מידע עם שכבת גושי קדסטר מודרנית

In [ ]:
unreg_parcels[['GUSH_SUFFIX', 'GUSH_NUM']] = unreg_parcels[['GUSH_SUFFIX', 'GUSH_NUM']].fillna(0)
unreg_parcels[['GUSH_SUFFIX', 'GUSH_NUM']] = unreg_parcels[['GUSH_SUFFIX', 'GUSH_NUM']].astype(int)
unreg_parcels['BlockName'] = unreg_parcels['GUSH_NUM'].astype(str) + '_' + unreg_parcels['GUSH_SUFFIX'].astype(str)

Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

unreg_parcels = unreg_parcels.merge(Blocks, on = 'BlockName', how = 'left')
unreg_parcels['BlockUniqueID'] = unreg_parcels['BlockUniqueID'].astype('Int64')

unreg_parcels.drop(columns=['GUSH_NUM', 'GUSH_SUFFIX', 'BlockName', 'BlockNumber', 'SubBlockNumber'], inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של תהליך יוצר לפי תצ"ר בשטח לא מוסדר (4)
<h1 style="text-align:right;font-size:100%">
במקרים לא ידועים, התהליך היוצר הינו הסדר מקרקעין (1)

In [ ]:
unreg_parcels['CreateProcessType'] = unreg_parcels['ProcessType'].replace(11, 4)

unreg_parcels['CreateProcessType'] = unreg_parcels['CreateProcessType'].fillna(1)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  14.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("ParcelUniqueID", int),
                        ("BlockUniqueID", int),
                        ("ParcelNumber", int),
                        ("ParcelType", int),
                        ("CreateProcessType", str),
                        ("LegalArea", int),
                        ("LandDesignationPlan", int),
                        ("CPBUniqueCreateProcessID", int),
                        ("CPBUniqueUpdateProcessID", int),
                        ("CPBUniqueCancelProcessID", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])


UnregisteredParcels2D = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                         geometry = 'geometry',
                                         crs = 2039)

del new_columns

In [ ]:
UnregisteredParcels2D['ParcelUniqueID'] = unreg_parcels['Identifier'] + 5000000
UnregisteredParcels2D['ParcelUniqueID'] = unreg_parcels['Identifier'].astype(int)

UnregisteredParcels2D['BlockUniqueID'] = unreg_parcels['BlockUniqueID'].astype('Int64')

UnregisteredParcels2D['ParcelNumber'] = unreg_parcels['PARCEL'].astype(int)

UnregisteredParcels2D['ParcelType'] = 1 # ארעית

UnregisteredParcels2D['CreateProcessType'] = unreg_parcels['CreateProcessType'].astype(int)

UnregisteredParcels2D['LegalArea'] = unreg_parcels['LEGAL_AREA'] * 1000
UnregisteredParcels2D['LegalArea'] = unreg_parcels['LEGAL_AREA'].astype(float)

UnregisteredParcels2D['LandDesignationPlan'] = None

UnregisteredParcels2D['CPBUniqueCreateProcessID'] = unreg_parcels['CPBUniqueID'].fillna(0)
UnregisteredParcels2D['CPBUniqueCreateProcessID'] = UnregisteredParcels2D['CPBUniqueCreateProcessID'].astype('Int64')

UnregisteredParcels2D['CPBUniqueUpdateProcessID'] = 0

UnregisteredParcels2D['CPBUniqueCancelProcessID'] = 0

UnregisteredParcels2D['geometry'] = unreg_parcels['geometry']

In [ ]:
UnregisteredParcels2D.sort_values('ParcelUniqueID', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  14.e Exporting results to product folder')

In [ ]:
#csv_dir  = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\UnregisteredParcels2D.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\UnregisteredParcels2D.geojson'


#UnregisteredParcels2D.drop(columns='geometry').to_csv(csv_dir ,encoding='cp1255', index=False)
UnregisteredParcels2D.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\UnregisteredParcels2D-Save as Feature Class.bat"]);